In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import numpy as np
import pandas as pd

def rank_column(df, column, ascending=False):
    _df = df.copy()
    col_name = f"rank_{column}"
    if col_name not in _df.columns:
        _df[col_name] = np.nan
    _df.loc[_df[col_name].isna(), col_name] = _df.groupby("start_time")[column].rank(
        ascending=ascending
    )
    return _df    

def prepare_data(data: pd.DataFrame):
    """
    Prepare the data for the strategy
    """
    data = data.copy()
    window_size = 7 * 24 * 4  # Adjust this based on your exact data frequency
    prep_col = [
        "pct_change",
        "rolling_accumulated_pct_change",
        "rolling_variance_pct_change",
        "rank",
    ]
    for col in prep_col:
        if col not in data.columns:
            data[col] = np.nan
    ### pct chage
    data.loc[data["pct_change"].isna(), "pct_change"] = data.groupby(level=0)[
        "close"
    ].pct_change()
    ### rolling_acc_pct_change
    data.loc[
        data["rolling_accumulated_pct_change"].isna(),
        "rolling_accumulated_pct_change",
    ] = data.groupby(level=0)["pct_change"].transform(
        lambda x: (
            x.rolling(window=window_size, min_periods=1).apply(
                lambda y: np.prod(1 + y / 100)
            )
            - 1
        )
        * 100
    )
    data.loc[
        data["rolling_variance_pct_change"].isna(), "rolling_variance_pct_change"
    ] = data.groupby(level=0)["pct_change"].transform(
        lambda x: x.rolling(window=window_size, min_periods=1).var()
    )
    data = rank_column(data, "rolling_accumulated_pct_change", ascending=False)
    data = rank_column(data, "rolling_variance_pct_change", ascending=True)
    data.loc[data["rank"].isna(), "rank"] = (
        data["rank_rolling_accumulated_pct_change"]
        + data["rank_rolling_variance_pct_change"]
    ) / 2
    return data

In [3]:
from settings import DATA_DIR

df = pd.read_csv(DATA_DIR / "all_15m_2880.csv", parse_dates=[1, 2])
df.set_index(["symbol", "start_time"], inplace=True)
df.head()

datetime             end_time interval  \
symbol start_time                                                              
BTC    2024-03-27 00:00:00 2024-03-27 00:00:00  2024-03-27-00:14:59      15m   
       2024-03-27 00:15:00 2024-03-27 00:15:00  2024-03-27-00:29:59      15m   
       2024-03-27 00:30:00 2024-03-27 00:30:00  2024-03-27-00:44:59      15m   
       2024-03-27 00:45:00 2024-03-27 00:45:00  2024-03-27-00:59:59      15m   
       2024-03-27 01:00:00 2024-03-27 01:00:00  2024-03-27-01:14:59      15m   

                            number_of_trades    close     high      low  \
symbol start_time                                                         
BTC    2024-03-27 00:00:00               500  69959.0  70076.0  69886.0   
       2024-03-27 00:15:00               470  70175.0  70183.0  69897.0   
       2024-03-27 00:30:00              1014  70543.0  70589.0  70087.0   
       2024-03-27 00:45:00               844  70735.0  70795.0  70378.0   
       2024-03-27 01:00:00               980  70748.0  70800.0  70481.0   

                               open    volume            tic            toc  
symbol start_time                                                            
BTC    2024-03-27 00:00:00  70049.0  24.21455  1711497600000  1711498499999  
       2024-03-27 00:15:00  69959.0  12.35217  1711498500000  1711499399999  
       2024-03-27 00:30:00  70176.0  73.40334  1711499400000  1711500299999  
       2024-03-27 00:45:00  70540.0  75.26471  1711500300000  1711501199999  
       2024-03-27 01:00:00  70735.0  70.56705  1711501200000  1711502099999

In [4]:
ranked_df = prepare_data(df)


In [5]:
ranked_df.head()

datetime             end_time interval  \
symbol start_time                                                              
BTC    2024-03-27 00:00:00 2024-03-27 00:00:00  2024-03-27-00:14:59      15m   
       2024-03-27 00:15:00 2024-03-27 00:15:00  2024-03-27-00:29:59      15m   
       2024-03-27 00:30:00 2024-03-27 00:30:00  2024-03-27-00:44:59      15m   
       2024-03-27 00:45:00 2024-03-27 00:45:00  2024-03-27-00:59:59      15m   
       2024-03-27 01:00:00 2024-03-27 01:00:00  2024-03-27-01:14:59      15m   

                            number_of_trades    close     high      low  \
symbol start_time                                                         
BTC    2024-03-27 00:00:00               500  69959.0  70076.0  69886.0   
       2024-03-27 00:15:00               470  70175.0  70183.0  69897.0   
       2024-03-27 00:30:00              1014  70543.0  70589.0  70087.0   
       2024-03-27 00:45:00               844  70735.0  70795.0  70378.0   
       2024-03-27 01:00:00               980  70748.0  70800.0  70481.0   

                               open    volume            tic            toc  \
symbol start_time                                                             
BTC    2024-03-27 00:00:00  70049.0  24.21455  1711497600000  1711498499999   
       2024-03-27 00:15:00  69959.0  12.35217  1711498500000  1711499399999   
       2024-03-27 00:30:00  70176.0  73.40334  1711499400000  1711500299999   
       2024-03-27 00:45:00  70540.0  75.26471  1711500300000  1711501199999   
       2024-03-27 01:00:00  70735.0  70.56705  1711501200000  1711502099999   

                            pct_change  rolling_accumulated_pct_change  \
symbol start_time                                                        
BTC    2024-03-27 00:00:00         NaN                             NaN   
       2024-03-27 00:15:00    0.003088                        0.003088   
       2024-03-27 00:30:00    0.005244                        0.008332   
       2024-03-27 00:45:00    0.002722                        0.011054   
       2024-03-27 01:00:00    0.000184                        0.011237   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
BTC    2024-03-27 00:00:00                          NaN   NaN   
       2024-03-27 00:15:00                          NaN   NaN   
       2024-03-27 00:30:00                     0.000002  32.5   
       2024-03-27 00:45:00                     0.000002  22.0   
       2024-03-27 01:00:00                     0.000004  28.5   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
BTC    2024-03-27 00:00:00                                  NaN   
       2024-03-27 00:15:00                                 48.0   
       2024-03-27 00:30:00                                 42.0   
       2024-03-27 00:45:00                                 35.0   
       2024-03-27 01:00:00                                 44.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
BTC    2024-03-27 00:00:00                               NaN  
       2024-03-27 00:15:00                               NaN  
       2024-03-27 00:30:00                              23.0  
       2024-03-27 00:45:00                               9.0  
       2024-03-27 01:00:00                              13.0

In [6]:
def get_non_volatile_coins(df):
    # Group by the 'symbol' level of the MultiIndex and get the index of the max 'start_time' for each 'symbol'
    idx = df.groupby(level='symbol')['end_time'].idxmax()

    # Select the rows with the latest 'end_time' for each 'symbol'
    latest_df = df.loc[idx]

    # Assuming 'rank' is a column in your DataFrame, sort these rows by 'rank' in ascending order
    sorted_latest_df = latest_df.sort_values(by='rank_rolling_variance_pct_change', ascending=True)

    print("dropped:", sorted_latest_df.index.get_level_values('symbol').tolist()[-10:])
    return sorted_latest_df.index.get_level_values('symbol').tolist()[:-10]

good_coins = get_non_volatile_coins(ranked_df)
len(good_coins)

dropped: ['WLD', 'W', 'ONDO', 'BOME', 'AR', 'kFLOKI', 'kPEPE', 'MAVIA', 'WIF', 'kBONK']


47

In [7]:
ranked_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 164217 entries, ('BTC', Timestamp('2024-03-27 00:00:00')) to ('ENS', Timestamp('2024-04-26 00:00:00'))
Data columns (total 17 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   datetime                             164217 non-null  datetime64[ns]
 1   end_time                             164217 non-null  object        
 2   interval                             164217 non-null  object        
 3   number_of_trades                     164217 non-null  int64         
 4   close                                164217 non-null  float64       
 5   high                                 164217 non-null  float64       
 6   low                                  164217 non-null  float64       
 7   open                                 164217 non-null  float64       
 8   volume                               164217 non-null  float64       


In [8]:
cols = [
    "pct_change",
    "rolling_accumulated_pct_change",
    "rank_rolling_accumulated_pct_change",
    "rolling_variance_pct_change",
    "rank_rolling_variance_pct_change",
    "rank",
]
non_volatile_reranked_df = ranked_df.loc[good_coins].drop(cols, axis=1)
non_volatile_reranked_df = prepare_data(non_volatile_reranked_df)

In [9]:
non_volatile_reranked_df.head()

datetime             end_time interval  \
symbol start_time                                                              
BNB    2024-03-27 00:00:00 2024-03-27 00:00:00  2024-03-27-00:14:59      15m   
       2024-03-27 00:15:00 2024-03-27 00:15:00  2024-03-27-00:29:59      15m   
       2024-03-27 00:30:00 2024-03-27 00:30:00  2024-03-27-00:44:59      15m   
       2024-03-27 00:45:00 2024-03-27 00:45:00  2024-03-27-00:59:59      15m   
       2024-03-27 01:00:00 2024-03-27 01:00:00  2024-03-27-01:14:59      15m   

                            number_of_trades   close    high     low    open  \
symbol start_time                                                              
BNB    2024-03-27 00:00:00                45  579.57  581.05  578.55  580.63   
       2024-03-27 00:15:00                25  581.94  581.94  579.50  579.63   
       2024-03-27 00:30:00                20  583.30  583.30  581.40  581.68   
       2024-03-27 00:45:00                33  584.86  584.86  582.57  582.78   
       2024-03-27 01:00:00                54  585.49  585.86  584.16  584.78   

                             volume            tic            toc  pct_change  \
symbol start_time                                                               
BNB    2024-03-27 00:00:00   67.070  1711497600000  1711498499999         NaN   
       2024-03-27 00:15:00   16.187  1711498500000  1711499399999    0.004089   
       2024-03-27 00:30:00   28.878  1711499400000  1711500299999    0.002337   
       2024-03-27 00:45:00   63.013  1711500300000  1711501199999    0.002674   
       2024-03-27 01:00:00  131.356  1711501200000  1711502099999    0.001077   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
BNB    2024-03-27 00:00:00                             NaN   
       2024-03-27 00:15:00                        0.004089   
       2024-03-27 00:30:00                        0.006426   
       2024-03-27 00:45:00                        0.009101   
       2024-03-27 01:00:00                        0.010178   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
BNB    2024-03-27 00:00:00                          NaN   NaN   
       2024-03-27 00:15:00                          NaN   NaN   
       2024-03-27 00:30:00                 1.535151e-06  30.5   
       2024-03-27 00:45:00                 8.643032e-07  19.0   
       2024-03-27 01:00:00                 1.533060e-06  22.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
BNB    2024-03-27 00:00:00                                  NaN   
       2024-03-27 00:15:00                                 33.0   
       2024-03-27 00:30:00                                 42.0   
       2024-03-27 00:45:00                                 35.0   
       2024-03-27 01:00:00                                 39.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
BNB    2024-03-27 00:00:00                               NaN  
       2024-03-27 00:15:00                               NaN  
       2024-03-27 00:30:00                              19.0  
       2024-03-27 00:45:00                               3.0  
       2024-03-27 01:00:00                               5.0

In [10]:
non_volatile_reranked_df.index.get_level_values('symbol').nunique()

47

In [11]:
non_volatile_reranked_df.to_csv("ranked.csv")

In [12]:
non_volatile_reranked_df.head()

datetime             end_time interval  \
symbol start_time                                                              
BNB    2024-03-27 00:00:00 2024-03-27 00:00:00  2024-03-27-00:14:59      15m   
       2024-03-27 00:15:00 2024-03-27 00:15:00  2024-03-27-00:29:59      15m   
       2024-03-27 00:30:00 2024-03-27 00:30:00  2024-03-27-00:44:59      15m   
       2024-03-27 00:45:00 2024-03-27 00:45:00  2024-03-27-00:59:59      15m   
       2024-03-27 01:00:00 2024-03-27 01:00:00  2024-03-27-01:14:59      15m   

                            number_of_trades   close    high     low    open  \
symbol start_time                                                              
BNB    2024-03-27 00:00:00                45  579.57  581.05  578.55  580.63   
       2024-03-27 00:15:00                25  581.94  581.94  579.50  579.63   
       2024-03-27 00:30:00                20  583.30  583.30  581.40  581.68   
       2024-03-27 00:45:00                33  584.86  584.86  582.57  582.78   
       2024-03-27 01:00:00                54  585.49  585.86  584.16  584.78   

                             volume            tic            toc  pct_change  \
symbol start_time                                                               
BNB    2024-03-27 00:00:00   67.070  1711497600000  1711498499999         NaN   
       2024-03-27 00:15:00   16.187  1711498500000  1711499399999    0.004089   
       2024-03-27 00:30:00   28.878  1711499400000  1711500299999    0.002337   
       2024-03-27 00:45:00   63.013  1711500300000  1711501199999    0.002674   
       2024-03-27 01:00:00  131.356  1711501200000  1711502099999    0.001077   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
BNB    2024-03-27 00:00:00                             NaN   
       2024-03-27 00:15:00                        0.004089   
       2024-03-27 00:30:00                        0.006426   
       2024-03-27 00:45:00                        0.009101   
       2024-03-27 01:00:00                        0.010178   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
BNB    2024-03-27 00:00:00                          NaN   NaN   
       2024-03-27 00:15:00                          NaN   NaN   
       2024-03-27 00:30:00                 1.535151e-06  30.5   
       2024-03-27 00:45:00                 8.643032e-07  19.0   
       2024-03-27 01:00:00                 1.533060e-06  22.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
BNB    2024-03-27 00:00:00                                  NaN   
       2024-03-27 00:15:00                                 33.0   
       2024-03-27 00:30:00                                 42.0   
       2024-03-27 00:45:00                                 35.0   
       2024-03-27 01:00:00                                 39.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
BNB    2024-03-27 00:00:00                               NaN  
       2024-03-27 00:15:00                               NaN  
       2024-03-27 00:30:00                              19.0  
       2024-03-27 00:45:00                               3.0  
       2024-03-27 01:00:00                               5.0

In [13]:
non_volatile_reranked_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 135407 entries, ('BNB', Timestamp('2024-03-27 00:00:00')) to ('SEI', Timestamp('2024-04-26 00:00:00'))
Data columns (total 17 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   datetime                             135407 non-null  datetime64[ns]
 1   end_time                             135407 non-null  object        
 2   interval                             135407 non-null  object        
 3   number_of_trades                     135407 non-null  int64         
 4   close                                135407 non-null  float64       
 5   high                                 135407 non-null  float64       
 6   low                                  135407 non-null  float64       
 7   open                                 135407 non-null  float64       
 8   volume                               135407 non-null  float64       


In [14]:
# Group by the 'symbol' level of the MultiIndex and get the index of the max 'start_time' for each 'symbol'
idx = non_volatile_reranked_df.groupby(level='symbol')['end_time'].idxmax()

# Select the rows with the latest 'end_time' for each 'symbol'
latest_df = non_volatile_reranked_df.loc[idx]

# Assuming 'rank' is a column in your DataFrame, sort these rows by 'rank' in ascending order
sorted_latest_df = latest_df.sort_values(by='rank', ascending=True)


In [15]:
sorted_latest_df.loc[:, [ "end_time", "rank"]]

,,end_time,rank
symbol,start_time,,
BNB,2024-04-26,2024-04-26-00:14:59,4.0
XRP,2024-04-26,2024-04-26-00:14:59,11.5
LTC,2024-04-26,2024-04-26-00:14:59,12.5
ETH,2024-04-26,2024-04-26-00:14:59,13.5
MATIC,2024-04-26,2024-04-26-00:14:59,14.0
LINK,2024-04-26,2024-04-26-00:14:59,14.0
BTC,2024-04-26,2024-04-26-00:14:59,15.0
COMP,2024-04-26,2024-04-26-00:14:59,15.0
UNI,2024-04-26,2024-04-26-00:14:59,16.0


In [16]:
sorted_latest_df = sorted_latest_df.loc[:, [ "end_time", "rank"]]
sorted_latest_df

,,end_time,rank
symbol,start_time,,
BNB,2024-04-26,2024-04-26-00:14:59,4.0
XRP,2024-04-26,2024-04-26-00:14:59,11.5
LTC,2024-04-26,2024-04-26-00:14:59,12.5
ETH,2024-04-26,2024-04-26-00:14:59,13.5
MATIC,2024-04-26,2024-04-26-00:14:59,14.0
LINK,2024-04-26,2024-04-26-00:14:59,14.0
BTC,2024-04-26,2024-04-26-00:14:59,15.0
COMP,2024-04-26,2024-04-26-00:14:59,15.0
UNI,2024-04-26,2024-04-26-00:14:59,16.0


In [17]:
sorted_latest_df.to_csv("ranked.csv")